# Initial setup

In [15]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = "arn:aws:iam::941656036254:role/service-role/AmazonSageMaker-ExecutionRole-20210904T193230"

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sm-modelparallel-distribution-options'
print('Bucket:\n{}'.format(bucket))

Bucket:
sagemaker-us-east-1-941656036254


In [ ]:
#to stream TB data  from S3 location need to install 
!pip install tensorflow-io

In [ ]:
sagemaker.__version__   

In [10]:
data_url = "s3://sagemaker-us-east-1-941656036254/hymenoptera_data"

In [8]:
bucket

'sagemaker-us-east-1-941656036254'

## Train Script



In [ ]:
! pygmentize 1_sources/train_resnet.py

# Remote Mode

In [13]:

from sagemaker.pytorch import PyTorch

instance_type = 'ml.p2.xlarge'
instance_count = 1
job_name = "pytorch-tb-profiling-2"

estimator = PyTorch(
          entry_point="train_resnet_tb.py", # Pick your train script
          source_dir='1_sources',
          role=role,
          instance_type=instance_type,
          sagemaker_session=sagemaker_session,
          image_uri="763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.10.2-gpu-py38-cu113-ubuntu20.04-sagemaker",
          instance_count=instance_count,
          hyperparameters={
              "batch-size":16,
              "num-epochs":5,
              "input-size" : 224,
              "feature-extract":False,
              "tb-s3-url": f"s3://{bucket}/tensorboard/{job_name}",
              "num-data-workers": 4
          },
          disable_profiler=True,
          debugger_hook_config=False,
          base_job_name=job_name,
      )

In [14]:
estimator.fit(inputs={"train":f"{data_url}/train", "val":f"{data_url}/val"})

2022-05-28 16:09:49 Starting - Starting the training job...
2022-05-28 16:10:16 Starting - Preparing the instances for training.........
2022-05-28 16:11:39 Downloading - Downloading input data...
2022-05-28 16:12:20 Training - Downloading the training image.................................
2022-05-28 16:17:47 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-05-28 16:17:51,135 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-28 16:17:51,165 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-28 16:17:51,174 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-05-28 16:17:51,793 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.8 -m pip install -r requirements.txt
━━━━━━━━━━━━